# LangChain Agents!

NOTE: You will need a Google Search API Key.
You can get a key for free from https://serpapi.com/

Make sure that you have an environment variable named: SERPAPI_API_KEY
with the value being the value of your key

In [ ]:
pip -q install langchain openai google-search-results tiktoken wikipedia numexpr

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [4]:
llm = OpenAI(temperature=0)

/Users/justin/dev/HALICON-2024/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
tools = load_tools(["serpapi"], llm=llm)

In [7]:
# What is a tool like this
tools[0].name, tools[0].description

('Search',
 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.')

In [9]:
agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True,
                         handle_parsing_errors=True)

/Users/justin/dev/HALICON-2024/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
# (ReAct) = Reasoning and Acting which follows a Chain of Thought (CoT) prompting strategy which leads to to better performance

# take a look at the template!
# NOTE!!!! : The agent already has built up a prompt template for us based on the description of the tool
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

# Chain Of Thought

Running these question on the agents will cause it to get through a series of observations and thoughts.
See below!

![image.png](react-prompting2.png)

In [11]:

agent.run("How can I treat a sprained ankle?")


/Users/justin/dev/HALICON-2024/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Search
Action Input: "treatment for sprained ankle"
Observation: ['Sprained ankle entity_type: medicine, health.', 'Sprained ankle main_tab_text: Overview.', 'Sprained ankle kgmid: /m/025tl48.', 'The treatment goals are to reduce pain and swelling, promote healing of the ligament, and restore function of the ankle.', 'Apply ice to the sore part of your ankle, typically for 15 minutes on and then 15 minutes off, as much as possible for the rest of the day.', 'This usually means adopting the classic RICE regimen — rest, ice, compression, and elevation. If you have severe pain and swelling, rest your ...', 'Nonsteroidal anti-inflammatory medication, such as ibuprofen (Motrin® or Advil®) is used to reduce swelling. Take these as directed by your health care provider ...', 'How to treat sprains and strains yourself · Rest – stop any exercise or activities and try not to put any weight on the injury. · 

'The treatment for a sprained ankle includes rest, ice, compression, and elevation, as well as taking nonsteroidal anti-inflammatory medication. It is important to also see a therapist for proper assessment and treatment.'

In [10]:
agent.run("site:webmd.com How can I treat a sprained ankle?")



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Search
Action Input: site:webmd.com How can I treat a sprained ankle?
Observation: ['Ice the ankle for 20 to 30 minutes every 2 to 3 hours for the first 2 days; Compress (wrap) the ankle lightly -- not tightly -- ...', 'The stages of therapy may include: The therapist assesses your condition and will provide treatment to reduce pain and swelling.', "Typically, the initial treatment of a sprain includes resting and protecting the ankle until swelling goes down for about 1 week. That's ...", 'Control Swelling With RICE Therapy\u200b\u200b RICE stands for: Rest the sprained or strained area. If necessary, use a sling for an arm injury or crutches ...', 'RICE is a simple, at-home treatment for sprains and strains. Learn how rest, ice, elevation, and compression can help you reduce pain and ...', 'Towel curls. Sit in a chair, with your injured foot flat on a towel on the floor. Curl your toes to grab t

'The best way to treat a sprained ankle is to follow the RICE method (Rest, Ice, Compression, Elevation) and to avoid putting weight on the injured area for 24-48 hours. It is also recommended to see a therapist for further treatment and to avoid invasive treatments such as surgery unless absolutely necessary.'

# Smarter Agent

Let's make an even smarter agent that can do more things!

In [11]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [12]:
agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True,
                         handle_parsing_errors=True)

In [13]:
# take a look at the template!
# NOTE!!!! : The agent already has built up a prompt template for us based on the description of the tool
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [20]:
agent.run("site:webmd.com How many days does it take to recover from a sprained ankle divided in half?")



> Entering new AgentExecutor chain...
 I should use a search engine to find the answer on a reliable medical website.
Action: Search
Action Input: site:webmd.com How many days does it take to recover from a sprained ankle divided in half?
Observation: ["To help your sprained ankle heal, you'll ... Ice the ankle for 20 to 30 minutes every 2 to 3 hours for the first 2 days ... Each layer should cover ...", 'Ligaments and tendons can take longer to heal after a fracture is fully mended. It can take as long as 2 years to completely recover full ...', 'It can take several weeks for a broken leg to heal fully. The time it takes depends on how bad the break was and how healthy you are. Younger ...', 'Take 1 to 2 days ... Most workout injuries will heal on their own in 4 weeks or less. ... For example, if you sprain your ankle, exercise your arms ...', 'Ice wrapped in a small towel and applied to the injured foot may also reduce swelling and pain for the first several hours after an injury. 

'On average, it takes 5 days to recover from a sprained ankle divided in half.'